# asf_search - Basic Overview
`asf_search` provides several helpful classes and functions to simplify working with the ASF catalog. This notebook briefly describes how to get started with `asf_search`.
***
## Before You Start
  
The steps outlined in this notebook assume `asf_search` is available on your system. `asf_search` is available through [PyPi](https://pypi.org/project/asf-search/), [Conda](https://anaconda.org/conda-forge/asf_search), and [Github](https://github.com/asfadmin/Discovery-asf_search). Additionally, full documentation for `asf_search` and many other services offered by ASF is available at [https://docs.asf.alaska.edu/](https://docs.asf.alaska.edu/)
  
For this demonstration, we have already installed `asf_search` within a virtual environment through PyPi via the command:
  
  
```pip install asf_search```
  
`asf_search` requires Python 3.6 or higher.

In [2]:
!pip install asf_search

/Users/ethanhsiao/opt/anaconda3/lib/python3.7/site-packages/cryptography/hazmat/backends/openssl/x509.py:18: CryptographyDeprecationWarning: This version of cryptography contains a temporary pyOpenSSL fallback path. Upgrade pyOpenSSL now.
  utils.DeprecatedIn35,
     |████████████████████████████████| 1.2 MB 3.4 MB/s eta 0:00:01
     |████████████████████████████████| 127 kB 2.5 MB/s eta 0:00:01
ERROR: rasa 2.8.9 has requirement numpy<1.20.0,>=1.19.2, but you'll have numpy 1.21.5 which is incompatible.
ERROR: rasa 2.8.9 has requirement ruamel.yaml<0.17.0,>=0.16.5, but you'll have ruamel-yaml 0.15.87 which is incompatible.
ERROR: rasa-sdk 2.8.2 has requirement urllib3<2.0.0,>=1.26.5, but you'll have urllib3 1.25.11 which is incompatible.
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.26.7
    Uninstalling urllib3-1.26.7:
      Successfully uninstalled urllib3-1.26.7
/Users/ethanhsiao/opt/anaconda3/lib/python3.7/site-packages/cryptography/hazmat/backends/opens

***
## Usage
Once installed, simply import `asf_search` as you would any other Python module:

In [3]:
import asf_search as asf

`asf_search` version numbers are based on [Semantic Versioning 2.0.0](https://semver.org/spec/v2.0.0.html):

In [4]:
asf.__version__

'3.0.6'

***

## Performing a Basic Search
[View this search in Vertex](https://search.asf.alaska.edu/#/?resultsLoaded=true&dataset=SENTINEL-1)

For this basic example, we will specify two search parameters: a platform, and how many results we want to retrieve:
- Sentinel-1
- 5 results max
    - (results are retrieved newest-first, so this will be the 5 newest products).

In [3]:
results = asf.search(platform=asf.PLATFORM.SENTINEL1, maxResults=5)

Alternatively, it may be useful to handle your search arguments as a dictionary:

In [4]:
opts = {
    'platform': asf.PLATFORM.SENTINEL1,
    'maxResults': 5
}
results = asf.search(**opts)

Note the use of `asf_search`-provided constants, as many editors support use of these through autocompletion. Categories of constants include:
- `BEAMMODE`
- `FLIGHT_DIRECTION`
- `INSTRUMENT`
- `PLATFORM`
- `POLARIZATION`
- `PRODUCT_TYPE`
  


***
## Working With Results
### `ASFSearchResults`
Search results are returned as an `ASFSearchResults` object, a subclass of `UserList`, containing a list of `ASFProduct` objects, each of these classes providing some additional functionality to aid in working with the results and individual products:

In [5]:
results

ASFSearchResults([<asf_search.ASFProduct.ASFProduct at 0x7f98e2fe20b0>,
                  <asf_search.ASFProduct.ASFProduct at 0x7f98e1bb2a40>])

### `ASFProduct`
`ASFProduct` provides a number of metadata fields, such as:
- Geographic coordinates
    - Latitude/Longitude
    - Shape type
- Scene and product metadata
    - Path, frame
    - Platform, beam, polarization
    - File name, size, URL
    - and many others
  
Geographic coordinates are stored in the `geometry` attribute:

In [6]:
results[0].geometry

{'coordinates': [[[-93.801613, -65.082001],
   [-91.983826, -66.146187],
   [-100.096245, -68.17202],
   [-101.686348, -67.025314],
   [-93.801613, -65.082001]]],
 'type': 'Polygon'}

Other metadata is available through the `properties` attribute:

In [7]:
results[0].properties

{'beamModeType': 'EW',
 'browse': ['https://datapool.asf.alaska.edu/BROWSE/SA/S1A_EW_GRDM_1SSH_20220330T022756_20220330T022817_042544_05132D_3C23.jpg'],
 'bytes': '43569636',
 'centerLat': '-66.659',
 'centerLon': '-96.7481',
 'faradayRotation': None,
 'fileID': 'S1A_EW_GRDM_1SSH_20220330T022756_20220330T022817_042544_05132D_3C23-GRD_MS',
 'fileName': 'S1A_EW_GRDM_1SSH_20220330T022756_20220330T022817_042544_05132D_3C23.zip',
 'flightDirection': 'ASCENDING',
 'frameNumber': '952',
 'granuleType': 'SENTINEL_1A_FRAME',
 'groupID': 'S1A_EWSH_0952_0956_042544_122',
 'insarStackId': None,
 'md5sum': '8a3a876115a4743e01f0464010766c9d',
 'offNadirAngle': None,
 'orbit': '42544',
 'pathNumber': '122',
 'platform': 'Sentinel-1A',
 'pointingAngle': None,
 'polarization': 'HH',
 'processingDate': '2022-03-30T02:27:56.000000',
 'processingLevel': 'GRD_MS',
 'sceneName': 'S1A_EW_GRDM_1SSH_20220330T022756_20220330T022817_042544_05132D_3C23',
 'sensor': 'C-SAR',
 'startTime': '2022-03-30T02:27:56.0000

### Output
The layout of the above data structure mirrors the geojson output format provided by ASF's SearchAPI, for a smooth transition to `asf_search`.
  
In fact, the `ASFSearchResults.__str__()` method serializes results to geojson _identical_ to that of ASF's SearchAPI, allowing for a drop-in replacement for users of the SearchAPI, simply by explicitly or implicitly casting search results to a string:

In [8]:
print(results)

{
  "features": [
    {
      "geometry": {
        "coordinates": [
          [
            [
              -93.801613,
              -65.082001
            ],
            [
              -91.983826,
              -66.146187
            ],
            [
              -100.096245,
              -68.17202
            ],
            [
              -101.686348,
              -67.025314
            ],
            [
              -93.801613,
              -65.082001
            ]
          ]
        ],
        "type": "Polygon"
      },
      "properties": {
        "beamModeType": "EW",
        "browse": [
          "https://datapool.asf.alaska.edu/BROWSE/SA/S1A_EW_GRDM_1SSH_20220330T022756_20220330T022817_042544_05132D_3C23.jpg"
        ],
        "bytes": "43569636",
        "centerLat": "-66.659",
        "centerLon": "-96.7481",
        "faradayRotation": null,
        "fileID": "S1A_EW_GRDM_1SSH_20220330T022756_20220330T022817_042544_05132D_3C23-GRD_MS",
        "fileName": "S1A_EW_G

Additionally, individual `ASFProduct` objects provides geojson-based serialization, in the form of a geojson `Feature` snippet:

In [9]:
print(results[0])

{
  "geometry": {
    "coordinates": [
      [
        [
          -93.801613,
          -65.082001
        ],
        [
          -91.983826,
          -66.146187
        ],
        [
          -100.096245,
          -68.17202
        ],
        [
          -101.686348,
          -67.025314
        ],
        [
          -93.801613,
          -65.082001
        ]
      ]
    ],
    "type": "Polygon"
  },
  "properties": {
    "beamModeType": "EW",
    "browse": [
      "https://datapool.asf.alaska.edu/BROWSE/SA/S1A_EW_GRDM_1SSH_20220330T022756_20220330T022817_042544_05132D_3C23.jpg"
    ],
    "bytes": "43569636",
    "centerLat": "-66.659",
    "centerLon": "-96.7481",
    "faradayRotation": null,
    "fileID": "S1A_EW_GRDM_1SSH_20220330T022756_20220330T022817_042544_05132D_3C23-GRD_MS",
    "fileName": "S1A_EW_GRDM_1SSH_20220330T022756_20220330T022817_042544_05132D_3C23.zip",
    "flightDirection": "ASCENDING",
    "frameNumber": "952",
    "granuleType": "SENTINEL_1A_FRAME",
    "g

## Summary
A complete example, showing how simple use of `asf_search` can be:

In [10]:
import asf_search as asf

print(asf.search(platform='S1', maxResults=5))

{
  "features": [
    {
      "geometry": {
        "coordinates": [
          [
            [
              -93.801613,
              -65.082001
            ],
            [
              -91.983826,
              -66.146187
            ],
            [
              -100.096245,
              -68.17202
            ],
            [
              -101.686348,
              -67.025314
            ],
            [
              -93.801613,
              -65.082001
            ]
          ]
        ],
        "type": "Polygon"
      },
      "properties": {
        "beamModeType": "EW",
        "browse": [
          "https://datapool.asf.alaska.edu/BROWSE/SA/S1A_EW_GRDM_1SSH_20220330T022756_20220330T022817_042544_05132D_3C23.jpg"
        ],
        "bytes": "43569636",
        "centerLat": "-66.659",
        "centerLon": "-96.7481",
        "faradayRotation": null,
        "fileID": "S1A_EW_GRDM_1SSH_20220330T022756_20220330T022817_042544_05132D_3C23-GRD_MS",
        "fileName": "S1A_EW_G

***
Next: [Geographic Searches](./2-Geographic_Search.ipynb)